# Word Embeddings

Natürliche Sprachen bieten uns mannigfaltige Möglichkeiten, dieselben Inhalte auf unterschiedliche Art und Weise auszudrücken. Wenn wir mit Texten arbeiten, reicht es daher in der Regel nicht aus, morphologische und syntaktische Strukturen zu betrachten, weil neben der Form und Anordnung der Wörter auch deren Bedeutung eine Rolle spielt. Als zusätzliche und wichtige Ebene kommt hier also die Semantik ins Spiel.

Die Abbildung von Wörtern auf Vektoren erlaubt es uns, mit diesen zu rechnen und zum Beispiel Distanzen oder Ähnlichkeiten zu bestimmen. Embeddings haben den Vorteil, dass sie eine Dimensionsreduktion mit sich bringen und semantische Embeddings sorgen darüber hinaus dafür, dass "verwandte" Wörter einen geringen Abstand voneinander haben.

Wir wollen uns im Folgenden zunächst mit Word2Vec beschäftigen und eine einfache Version des CBOW-Ansatzes selbst implementieren.

Danach schauen wir uns Gensim als Wrapping-Bibliothek für Wordembeddingmodelle an und werfen einen Blick auf Evaluationsmethoden für Embeddings sowie die ihnen inhärenten Biase.

## Aufgabe 1: Word2Vec CBOW
> You shall know a word by the company it keeps.
>
> -- <cite>J. R. Firth</cite>

Auf dem oben zitierten Prinzip beruhen die beiden als Word2Vec bekannt gewordenen Modelle CBOW und Skip Gram, die 2013 von [Tomas Mikolov et al.](https://arxiv.org/abs/1301.3781) bei Google entwickelt wurden.
Erstgenanntes Modell werden wir im Folgenden in einer einfachen Form selbst implementieren.

### 1.1 Trainingsdaten
Die Beschaffung und Aufbereitung von Trainingsdaten ist ein wichtiger Schritt in jeder NLP-Pipeline. Heute drücken wir uns davor und greifen auf das [`text8`-Datenset](http://mattmahoney.net/dc/textdata.html) zurück, das über den [Gensim-Downloader ](https://radimrehurek.com/gensim/downloader.html) heruntergeladen werden kann. Es besteht aus einem Auszug aus der Wikipedia und ist in der Gensim-Version bereits so vorbereitet, dass es als Liste von Wortlisten, den einzelnen Wikipediaartikeln, vorliegt.

Der Datensatz kann [hier](https://drive.google.com/drive/folders/1PKyQnB8Ox7QN7xdC8EjLCXZtDcpw90u3?usp=sharing) heruntergeladen werden. Ladet den Datensatz mit Pickle und wählt einen Teil der Daten (etwa 100 Artikel sollten zu Demonstrationszwecken genügen) als Testdatensatz aus.

In [1]:
import pickle

from google.colab import drive
drive.mount('/data')

# open a file, where you stored the pickled data
#file = open('data/text8', 'rb')
file = open('/data/My Drive/Colab Notebooks/data/text8', 'rb')

data = pickle.load(file) #TODO
data_sample = data[:100] #TODO

print(len(data_sample))

del data #nicht unbedingt nötig, aber wir verbrauchen eh schon so viel Speicher

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /data
100


### 1.2 Datenvorbereitung
Wir haben einen Datensatz, der aus Listen von Wörtern besteht. Unser Modell soll aber hinterher mit Zahlen hantieren und zwar entweder mit Wortindizes, die jedes Wort im Vokabular über einen eindeutige Nummer referenzierbar machen, oder mit One-hot-Vektoren, die als Labels dienen, mit denen der tatsächliche Output des Modells verglichen werden kann.

In [2]:
from collections import OrderedDict

# Beim Mapping von Wörtern zu IDs und umgekehrt sollte eine reproduzierbare Reihenfolge sichergestellt werden,
# um das Modell später weitertrainieren und die Embedding-Matrix interpretieren zu können.
# Diese Datenstruktur kann, aber muss nicht, als Basis dienen.
#unique_words = [OrderedDict.fromkeys(item) for item in data_sample] # Liste alle Wörter, die in unserem data_sample vorkommen)
unique_words = OrderedDict.fromkeys(item for sublist in data_sample for item in sublist)
#unique_words = OrderedDict.fromkeys(data_sample[0])
print(len(unique_words))

# Mapping von Wort zu ID

word2id = {word: index for index, word in enumerate(unique_words)} # TODO
#print(word2id)

# Mapping von ID zu Wort
id2word = {index: word for index, word in enumerate(unique_words)}# TODO
#print(id2word)

# Unser Data Sample, aber mit IDs statt Wörtern 
# [['der', 'hund', 'der', 'bellt'], ['die', 'katz', 'miaut']] => [[0, 1, 0, 2], [3, 4, 5]]
numeric_docs = [[word2id[w] for w in doc] for doc in data_sample]

print('Word to id sample:', list(word2id.items())[:10], '\n')
print('Id to word sample:', list(id2word.items())[:10], '\n')
print('Documents as lists of integers:', numeric_docs[0][:10])

52754
Word to id sample: [('anarchism', 0), ('originated', 1), ('as', 2), ('a', 3), ('term', 4), ('of', 5), ('abuse', 6), ('first', 7), ('used', 8), ('against', 9)] 

Id to word sample: [(0, 'anarchism'), (1, 'originated'), (2, 'as'), (3, 'a'), (4, 'term'), (5, 'of'), (6, 'abuse'), (7, 'first'), (8, 'used'), (9, 'against')] 

Documents as lists of integers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


Wir halten einige wichtige Parameter für unser Modell fest. Die Größe des Kontextfensters sowie die Länge der Embeddingvektoren können nach Bedarf angepasst werden. Für unsere Demo wählen wir kleine Werte.

In [3]:
vocabulary_size = len(word2id) # TODO
embedding_size = 50 # Länge der Embeddingvektoren
window_size = 2 # Größe des Kontextfensters. Wird nach rechts und links angewandt. Gesamter Kontext hier also 4 Wörter.

print('Vocabulary Size:', vocabulary_size)

Vocabulary Size: 52754


### 1.3 Generator
Um nicht alle Trainingsdaten auf einmal im Speicher halten zu müssen, schreiben wir uns eine Generatorfunktion, die Batches einer frei wählbaren Größe zurückgibt. Unser Ansatz ist dennoch nicht völlig speicherschonend, weil wir uns die Datengrundlage für die Generierung dieser Batches, nämlich die Integerlisten in `numeric_docs` sehr wohl im Speicher vorhalten. Darüber sehen wir aber großzügig hinweg.

Die Generatorfunktion erzeugt zwei numpy-Arrays der Länge `batch_size`, von denen das eine Listen mit Indizes der Kontexwörter enthält, die der Embedding-Layer als Eingabe erwartet, und das andere die zugehörigen One-Hot-Encodings der Mittelwörter.

Fiktives und vereinfachtes Beispiel:
<pre><code>* Fenstergröße: 1
* Batch-Size: 2
* Wortindizes: 'die': 0, 'ente': 1, 'lacht': 2, 'und': '3, 'quakt': 4 (und damit Vokabulargröße 5)
* Korpus (Auszug): [['die', 'ente', 'lacht', 'und', 'quakt'], ...]


=> Rückgabe: [[0, 2], [1, 3]], [[0, 1, 0, 0, 0], [0, 0, 1, 0, 0]]
</code></pre>

In [4]:
from keras.preprocessing import sequence
from keras.utils import np_utils
import numpy as np


def generate_context_word_batches(corpus, window_size, vocab_size, batch_size):
    X = []
    Y = []
    current_size = window_size
    context_length = window_size * 2
    
    while True:
      for sentence in corpus:
        X = []
        Y = []
        current_size = window_size
        sentence_length = len(sentence)
        for _ in range(batch_size):
          context_words = []
          target_word = None
          start = -window_size
          end = window_size + 1
          for j in range(start, end):
            if j != 0:
              context_words.append(sentence[current_size + j])
            else:
              target_word = sentence[current_size + j]
          # print(context_words)
          X.append(np.asarray(context_words))
          #print(np_utils.to_categorical(target_word, sentence_length))
          Y.append(np_utils.to_categorical(target_word, vocab_size))
          current_size += 1
        yield np.array(X), np.array(Y) # zwei numpy arrays


Using TensorFlow backend.


In [5]:
# Schneller Test
test_batch_size = 3
test_window_size = 2

print(numeric_docs[1])
print(id2word)
# test = [['die', 'ente', 'lacht', 'und', 'quakt'], ['the', 'old', 'testament', 'of', 'the', 'king', 'james', 'version', 'of', 'the', 'bible']]
gen = generate_context_word_batches(corpus=numeric_docs, window_size=test_window_size, vocab_size=vocabulary_size, batch_size=test_batch_size)
# for i in range(0, 2):
#   x, y = next(gen)
#   for j in range(0, test_batch_size):
#     print('Context (X):', [w for w in x[j]], '-> Target (Y):', y[j])
# print("x: \n", x)
# print("y: \n", y)
for i in range(0, 3): 
    x, y = next(gen)
    for j in range(0, test_batch_size):
        print('Context (X):', [id2word[w] for w in x[j]], '-> Target (Y):', id2word[np.argwhere(y[j])[0][0]])
print("x: \n", x)
print("y: \n", y)

[2520, 2505, 2521, 26, 1931, 2, 2507, 47, 218, 1691, 196, 5, 15, 1809, 2077, 26, 93, 2503, 467, 5, 15, 896, 5, 2323, 500, 89, 2522, 47, 126, 1577, 29, 2523, 419, 1494, 1683, 5, 1931, 331, 2, 2524, 93, 2525, 26, 110, 133, 2526, 2246, 2508, 2506, 26, 15, 1932, 29, 2111, 93, 2527, 3, 2329, 133, 141, 2073, 19, 2072, 87, 5, 500, 93, 2528, 500, 467, 5, 2529, 2513, 1237, 119, 1983, 93, 77, 2530, 1983, 2096, 29, 2064, 2238, 2071, 2072, 19, 2073, 1933, 5, 1934, 1531, 19, 2531, 2, 2507, 47, 218, 1691, 196, 5, 15, 1809, 2532, 2421, 133, 196, 93, 300, 2073, 19, 2071, 1933, 5, 369, 33, 24, 1930, 2112, 26, 2533, 93, 1166, 2534, 2535, 2536, 29, 1694, 2537, 2205, 93, 2538, 2073, 19, 2072, 2247, 2539, 1750, 780, 1699, 93, 2121, 2396, 93, 2540, 93, 379, 2541, 2234, 78, 2420, 2421, 133, 2542, 5, 2125, 1980, 93, 1930, 2543, 26, 218, 1691, 196, 5, 15, 1809, 1052, 133, 1984, 797, 29, 1633, 254, 459, 196, 77, 1687, 445, 500, 2, 8, 26, 77, 1931, 93, 254, 1981, 93, 1982, 1983, 15, 2544, 24, 89, 2450, 2545, 315

In [6]:
print(id2word[2520])
print(id2word[2505])
print(id2word[2521])
print(id2word[26])
print(id2word[1931])
print(id2word[2])

reciprocity
qualitative
impairments
in
communication
as


### 1.4 Definition des Models
Als nächstes definieren wir unser Model. Dazu verwenden wir die Sequential API von Keras.

In [7]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

#Modelldefinition
cbow = Sequential()
cbow.add(Embedding(input_dim=vocabulary_size, output_dim=embedding_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_size,)))
cbow.add(Dense(vocabulary_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Zusammenfassung
print(cbow.summary())






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 50)             2637700   
_________________________________________________________________
lambda_1 (Lambda)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 52754)             2690454   
Total params: 5,328,154
Trainable params: 5,328,154
Non-trainable params: 0
_________________________________________________________________
None


### 1.5 Training
Jetzt wird es ernst: Wir trainieren unser Modell.
Da wir eine eigene Generatorfunktion verwenden, müssen wir `steps_per_epoch` angeben. Dies ist die Anzahl der Generatoraufrufe pro Epoche. An sich ist es sinnvoll, diesen Wert auf `#samples//batch_size` zu setzen, damit das Modell pro Epoche alle Trainingsdaten sieht, aber weil wir Zeit sparen wollen, wählen wir einen geringeren Wert.

Weil wir unser Modell gerne abspeichern möchten, zum Beispiel, um es später weiter zu trainieren, definieren wir eine Callback-Funktion, die das für uns übernimmt.

In [0]:
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('embeddings.hd5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False)
   

In [9]:
epochs = 3
batch_size = 300
# Sollte eigentlich eher etwas in der Art von sum([len(doc) - 2 * window_size for doc in numeric_docs])//batch_size sein, aber s. o.
steps_per_epoch = 500 


cbow.fit_generator(
    generate_context_word_batches(corpus=numeric_docs, window_size=window_size, vocab_size=vocabulary_size, batch_size=batch_size),
    callbacks=[model_checkpoint],
    steps_per_epoch=steps_per_epoch,
    epochs=epochs
)    

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/3





500/500 [==============================] - 24s 48ms/step - loss: 9.2925

Epoch 00001: loss improved from inf to 9.29246, saving model to embeddings.hd5
Epoch 2/3
500/500 [==============================] - 19s 37ms/step - loss: 7.3206

Epoch 00002: loss improved from 9.29246 to 7.32056, saving model to embeddings.hd5
Epoch 3/3
500/500 [==============================] - 19s 38ms/step - loss: 6.9935

Epoch 00003: loss improved from 7.32056 to 6.99349, saving model to embeddings.hd5


### 1.6 Test
Nachdem wir unser Modell nur sehr kurz und nur auf wenigen Daten trainiert haben, ist davon auszugehen, dass die Ergebnisse nicht optimal sind. Einen kurzen Blick wollen wir dennoch riskieren.

Dazu extrahieren wir zunächst die Gewichte aus dem Embedding-Layer und schauen sie uns auszugsweise an.

In [10]:
import pandas as pd
from keras.models import load_model

cbow = load_model('embeddings.hd5') # TODO: Model laden
embedding_weights = cbow.get_weights()[0] # TODO: Auf Embedding Layer (1. Layer) des Modells zugreifen und dort die Gewichtsmatrix extrahieren
print(embedding_weights)
pd.DataFrame(embedding_weights, index=list(id2word.values())).head()

[[-0.05402246 -0.05578142  0.0646771  ...  0.13722615 -0.11528154
  -0.10080778]
 [-0.00257076 -0.06378445  0.06237193 ...  0.03293023 -0.00908771
  -0.04791247]
 [-0.36076164 -0.17751914  0.20021775 ... -0.00807526 -0.14751285
  -0.3185044 ]
 ...
 [ 0.02038956 -0.00364661 -0.03387836 ... -0.00180657 -0.00706597
  -0.01102257]
 [-0.00463321 -0.00478826 -0.0271266  ...  0.01405796 -0.03657931
   0.03232331]
 [-0.0347878  -0.03196237 -0.00515977 ...  0.0004063  -0.039621
   0.00253178]]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
anarchism,-0.054022,-0.055781,0.064677,0.050007,-0.101435,0.095980,0.141659,0.121086,0.114159,-0.049755,-0.090253,0.064249,0.127668,-0.096301,-0.066899,0.095754,0.104709,0.055633,0.117979,0.085651,-0.136476,0.094525,-0.064422,-0.118655,0.089123,-0.045990,0.055296,0.046061,0.046313,0.107965,-0.068662,-0.055991,-0.141399,0.135643,-0.114260,0.088712,-0.127674,-0.086227,-0.063107,-0.140984,0.128228,0.088646,-0.127971,0.108051,-0.102161,-0.107675,-0.062059,0.137226,-0.115282,-0.100808
originated,-0.002571,-0.063784,0.062372,0.029771,-0.071065,0.014877,0.071040,0.012151,0.005065,-0.069825,-0.089097,0.037900,0.029675,-0.041772,-0.092012,0.028064,0.043326,0.018604,0.039741,0.083107,-0.004385,0.060122,-0.011170,-0.073263,0.090639,-0.096587,0.034470,0.043422,0.013696,0.072354,-0.003269,0.002577,-0.019419,0.077921,-0.014183,0.027073,-0.035884,-0.027648,-0.065522,-0.088308,0.092466,0.076641,-0.087652,-0.004086,-0.087573,-0.068269,-0.061221,0.032930,-0.009088,-0.047912
as,-0.360762,-0.177519,0.200218,0.297954,0.051314,0.072269,-0.122929,-0.066077,0.090356,0.225408,0.039417,-0.151911,-0.088322,-0.130385,-0.105031,0.196034,0.186047,0.244348,-0.076282,0.264376,-0.270241,0.093937,-0.237160,-0.238611,0.416520,-0.032895,0.270805,0.190889,0.239593,0.080086,-0.178379,-0.294003,-0.039478,0.162188,-0.200405,0.059800,-0.205969,-0.130624,-0.199509,-0.289376,0.135588,0.068800,-0.284971,0.230223,0.061787,-0.036562,0.033925,-0.008075,-0.147513,-0.318504
a,-0.412226,-0.328697,-0.036524,0.029527,-0.285215,0.208456,0.039461,0.023727,0.284208,-0.568513,-0.050698,0.407167,0.202968,-0.301874,-0.105615,-0.028223,0.090964,0.323606,0.266030,0.220350,-0.047304,-0.024000,-0.004202,-0.112002,0.014629,0.021879,0.176038,0.040550,0.068485,-0.137714,-0.250171,-0.161928,-0.238763,0.103474,-0.111704,0.104478,-0.051088,-0.123623,0.174662,0.085912,-0.097484,-0.064328,-0.382827,0.042858,0.010403,-0.084735,-0.091638,0.113393,0.004631,-0.056492
term,-0.236669,-0.151769,0.164959,0.227837,-0.225462,0.153699,0.158521,0.164634,0.195015,-0.077387,-0.215407,0.073749,0.160004,-0.142714,-0.220949,0.243546,0.228084,0.237802,0.189095,0.251827,-0.149838,0.145713,-0.174664,-0.141098,0.252835,-0.132989,0.227932,0.226315,0.199488,0.203542,-0.041289,-0.187525,-0.238169,0.154828,-0.234042,0.228069,-0.124518,-0.202625,-0.219968,-0.251511,0.128787,0.246929,-0.223895,0.173188,-0.170757,-0.250906,-0.187127,0.105985,-0.214794,-0.209292


Da durch scharfes Hinsehen nicht unmittelbar zu erkennen ist, wie gut unsere Embeddings schon sind, machen wir stichprobenartige Tests. Dazu wählen wir einige Wörter und berechnen für deren Embeddings die Ähnlichkeit mit allen anderen Embedding-Vektoren in unserer Gewichtsmatrix. Anschließend lassen wir uns die fünf ähnlichsten Wörter ausgeben.
Als Distanzmaß wählen wir die Kosinusdistanz, die auf der Kosinusähnlichkeit beruht.

In [36]:
from sklearn.metrics.pairwise import cosine_distances

sample_terms = ['thus', 'influence', 'bible', 'climate', 'revolution', 'unix', 'term', 'working', 'topic', 'opinion']
sample_embeddings = [embedding_weights[word2id[term]] for term in sample_terms] # TODO
# print(sample_embeddings)
# Berechne die paarweisen Distanzen zwischen Beispielwörtern und Gesamtvokabular
distance_matrix = cosine_distances(sample_embeddings, embedding_weights)
print(distance_matrix)
print(distance_matrix.shape)

[[0.07213861 0.1697259  0.338638   ... 1.2020986  1.1959113  0.97691554]
 [0.20353895 0.28172213 0.47269142 ... 1.1474329  1.0655376  0.89761907]
 [0.0852055  0.20037639 0.34268332 ... 1.2300956  1.1799325  1.0295932 ]
 ...
 [0.12415624 0.22708803 0.3581019  ... 1.1515212  1.1741943  1.0557845 ]
 [1.0911069  1.1206073  1.2052284  ... 1.228013   1.0156057  1.0498955 ]
 [0.10023439 0.26318794 0.4872375  ... 1.0967635  1.2955885  1.1329745 ]]
(10, 52754)


In [12]:
# Zeige die top fünf ähnlichsten Wörter zu unseren Beispielwörtern 
similar_words = {sample_term: [id2word[idx] for idx in distance_matrix[index].argsort()[1:6]] 
                   for index, sample_term in enumerate(sample_terms)}

similar_words

{'bible': ['time', 'land', 'population', 'north', 'part'],
 'climate': ['european', 'coast', 'go', 'nation', 'scientists'],
 'influence': ['entered', 'ends', 'art', 'there', 'complete'],
 'opinion': ['actress', 'country', 'main', 'museums', 'letter'],
 'revolution': ['off', 'church', 'president', 'began', 'true'],
 'term': ['same', 'between', 'fact', 'part', 'across'],
 'thus': ['small', 'side', 'united', 'part', 'invasion'],
 'topic': ['desperation', 'beatrix', 'wager', 'benjedid', 'avalanche'],
 'unix': ['term', 'largest', 'introduced', 'under', 'lipid'],
 'working': ['president', 'so', 'province', 'say', 'covered']}

## Aufgabe 2: Gensim als Wrapper für Word2Vec-Modelle
Embedding-Layer begegnen einem in der Praxis in der Tat häufig. In der Regel aber nicht als Bestandteile von reinen Word Embedding-Trainingsmodellen, sondern als erster Layer für Modelle mit anderen Aufgaben. Die Embeddings werden dann entweder mit vorberechneten Werten initalisiert oder werden im Training des Modells für die Downstream-Aufgabe (Textklassifikation, Übersetzung, ...) mittrainiert.

Eine komfortable Möglichkeit, eigene Word2Vec-Modelle zu trainieren, bietet [Gensim](https://radimrehurek.com/gensim/models/word2vec.html), eine Bibliothek, die für diese Modelle auch Wrapper bereitstellt, um komfortabler an die Embeddings zu kommen und mit diesen zu arbeiten.

Wir wollen uns im Folgenden einen kleinen Ausschnitt der Möglichkeiten, die Gensim bietet, anschauen.

### 2.1 Word2Vec-Model laden
Wir werden mit vortrainierten Google-News-Embeddings arbeiten, die [hier](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing) heruntergeladen werden können. Die Vektoren haben die Länge 300.
Ladet die Embeddings über den oben angegebenen Link herunter und verwendet gensim, um sie zu laden.

**Hinweis**: Kann einen Moment dauern, bis das Dictionary, das von Wort auf Embedding abbildet, erzeugt ist. Im Zweifel ein ```limit``` angeben und nur die ersten 1,5 Mio. Embeddings laden.

In [13]:
import gensim
from gensim.models import KeyedVectors

embedding_file = open('/data/My Drive/Colab Notebooks/data/GoogleNews-vectors-negative300.bin.gz', 'rb')
embeddings = KeyedVectors.load_word2vec_format('/data/My Drive/Colab Notebooks/data/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=1500000)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Aus Spaß an der Freude können wir dann mal schauen, wie gut unser Modell ist bzw. ob es bestimmte von Menschen wahrgenommene Analogien bestätigt.

In [14]:
from gensim.test.utils import datapath

embeddings.evaluate_word_analogies(datapath("questions-words.txt"), restrict_vocab=30000)[0]

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.7708112206216831

### 2.2 Spaß mit Semantik
Im Folgenden wollen wir uns ein bisschen mit dem Mehrwert beschäftigen, den semantische Embeddings bieten. Weitere Inspiration zum Beispiel [hier](https://www.machinelearningplus.com/nlp/gensim-tutorial/) und in der [Doku](https://radimrehurek.com/gensim/models/keyedvectors.html).

Mit semantischen Vektoren lassen sich zum Beispiel folgende Fragen beantworten:


In [29]:
# Welche Stadt ist das New York Deutschlands? (Hinweis: 'New_York' ist als Token in den Embeddings enthalten)
result = embeddings.most_similar(positive=['Germany', 'New_York'])
#print(result)
print('Das deutsche New York ist: {}\n'.format(*result[0]))

# Was ist Emacs besonders ähnlich?
result = embeddings.similar_by_word("Emacs")
#print(result)
print('Ähnlichste Begriffe zu "Emacs": {}\n'.format(result))

# Und wie sieht es mit Vim aus?
result = embeddings.similar_by_word("Vim")
print('Ähnlichste Begriffe zu "Vim": {}\n'.format(result))

# Wer ist eigentlich der Mozart der Naturwissenschaft?
result = embeddings.most_similar(positive=['science', 'Mozart'])
# print(result)
print('Der Mozart der Naturwissenschaft ist: {}\n'.format(*result[0]))

# Welches Wort verhält sich zu 'singing' wie 'burnt' zu 'burning'?
#print('burning:burnt wie singing:{}\n'.format(# TODO))

# Sind sich Deutschland und Frankreich ähnlicher oder Deutschland und Kanada?
dist_de_fr = embeddings.distance("Germany", "France")
dist_de_can = embeddings.distance("Germany", "Canada")

sim_de_fr = embeddings.similarity("Germany", "France")
sim_de_can = embeddings.similarity("Germany", "Canada")

print('Distanz DE, FR: {}'.format(dist_de_fr))
print('Distanz DE, CAN: {}'.format(dist_de_can))

print('Ähnlichkeit DE, FR: {}'.format(sim_de_fr))
print('Ähnlichkeit DE, CAN: {}'.format(sim_de_can))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Das deutsche New York ist: Berlin

Ähnlichste Begriffe zu "Emacs": [('emacs', 0.7516424655914307), ('wget', 0.6600909233093262), ('TextMate', 0.6504663825035095), ('Gnumeric', 0.6487864255905151), ('Notepad_+', 0.6460206508636475), ('Win##', 0.6429656147956848), ('debian', 0.6395151019096375), ('libc', 0.6370412707328796), ('osx', 0.6365400552749634), ('commandline', 0.6359836459159851)]

Ähnlichste Begriffe zu "Vim": [('Emacs', 0.4973173439502716), ('deo', 0.47666671872138977), ('emacs', 0.46879321336746216), ('TextWrangler', 0.46436336636543274), ('Fantastik', 0.4486047625541687), ('TextMate', 0.44810065627098083), ('Chill_Pill', 0.4391757547855377), ('wget', 0.4389176666736603), ('Nisus_Writer_Pro', 0.437983900308609), ('freeware', 0.4375055134296417)]

Der Mozart der Naturwissenschaft ist: Beethoven

Distanz DE, FR: 0.3729243874549866
Distanz DE, CAN: 0.6192150413990021
Ähnlichkeit DE, FR: 0.6270756125450134
Ähnlichkeit DE, CAN: 0.3807849586009979


Bei der Interpretation der Ergebnisse ist jedoch Vorsicht geboten: Es werden zwar semantische Beziehungen abgebildet, aber die entsprechen möglicherweise nicht immer den Erwartungen.

Wie ähnlich sind sich zum Beispiel "Leben" und "Tod", "kalt" und "warm", "Norden" und "Süden"?

Sind die Ergebnisse wie erwartet? Warum (nicht)?

In [31]:
# TODO: Ähnlichkeiten berechnen
print("life <--> death: ", embeddings.similarity("life", "death"))
print("cold <--> warm: ", embeddings.similarity("cold", "warm"))
print("north <--> south: ", embeddings.similarity("north", "south"))

life <--> death:  0.36187765
cold <--> warm:  0.59530354
north <--> south:  0.96745354


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Die Embeddings sind nicht neutral, sondern spiegeln die Beziehungen wieder, die sich in den Trainingsdaten finden lassen.

In [35]:
# Wird Wissenschaft von Frauen oder Männern gemacht?
result = embeddings.distances('science', ['man', 'woman'])
print('Wissenschaft wird gemacht von: {}'.format(result))

# Sind Mörder eher Schwarze, Weiße oder Asiaten?
result = embeddings.similar_by_word("murderer")
print('Mörder sind: {}'.format(result))

# Was bleibt vom Mann, wenn die Intelligenz abgezogen wird?
result = embeddings.most_similar(positive=['man'], negative=['intelligence'])
print('Mann ohne Intelligenz: {}'.format(result))

Wissenschaft wird gemacht von: [0.9742661 0.9528713]
Mörder sind: [('rapist', 0.7380059957504272), ('murderers', 0.7061031460762024), ('serial_killer', 0.701305091381073), ('mass_murderer', 0.6719599962234497), ('murder', 0.6650081872940063), ('killer', 0.6645797491073608), ('cold_blooded_murderer', 0.6416785717010498), ('Murderer', 0.6403113007545471), ('psychopath', 0.6380615234375), ('convicted_murderer', 0.6339195966720581)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Mann ohne Intelligenz: [('woman', 0.525009274482727), ('boy', 0.4727959632873535), ('teenager', 0.4585689902305603), ('teenage_girl', 0.42294150590896606), ('girl', 0.3896019458770752), ('teen_ager', 0.3817293643951416), ('robber', 0.37824171781539917), ('Man', 0.3740370571613312), ('Robbery_suspect', 0.3724946975708008), ('motorcyclist', 0.36545804142951965)]
